# AutoGen आधारभूत नमूना

यस कोड नमूनामा, तपाईं [AutoGen](https://aka.ms/ai-agents/autogen) AI Framework प्रयोग गरेर एउटा आधारभूत एजेन्ट सिर्जना गर्नुहुनेछ।

यस नमूनाको उद्देश्य विभिन्न एजेन्टिक ढाँचाहरू कार्यान्वयन गर्दा पछि प्रयोग गरिने चरणहरू देखाउनु हो।


आवश्यक पाइथन प्याकेजहरू आयात गर्नुहोस्


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## क्लाइन्ट बनाउनुहोस्

यस नमुनामा, हामी [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) प्रयोग गर्नेछौं LLM पहुँचका लागि।

`model` लाई `gpt-4o-mini` भनेर परिभाषित गरिएको छ। GitHub Models बजारमा उपलब्ध अर्को मोडेलमा परिवर्तन गरेर फरक नतिजाहरू हेर्न प्रयास गर्नुहोस्।

छिटो परीक्षणको रूपमा, हामी केवल एउटा साधारण प्रश्न चलाउनेछौं - `What is the capital of France`।


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## एजेन्ट परिभाषित गर्दै 

अब हामीले `client` सेटअप गरिसकेपछि र यसको काम गर्ने पुष्टि गरेपछि, अब `AssistantAgent` सिर्जना गरौं। प्रत्येक एजेन्टलाई निम्न प्रकारका विशेषताहरू प्रदान गर्न सकिन्छ:  
**name** - छोटो नाम जसले बहु-एजेन्ट प्रवाहहरूमा यसलाई सन्दर्भ गर्न उपयोगी बनाउँछ।  
**model_client** - तपाईंले अघिल्लो चरणमा सिर्जना गरेको क्लाइन्ट।  
**tools** - एजेन्टले कुनै कार्य पूरा गर्न प्रयोग गर्न सक्ने उपलब्ध उपकरणहरू।  
**system_message** - मेटाप्रम्प्ट जसले LLM को कार्य, व्यवहार र स्वर परिभाषित गर्दछ।  

तपाईंले सिस्टम सन्देश परिवर्तन गरेर LLM कसरी प्रतिक्रिया दिन्छ हेर्न सक्नुहुन्छ। हामी `tools` को बारेमा पाठ #4 मा छलफल गर्नेछौं।


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## एजेन्ट चलाउनुहोस्

तलको फङ्सनले एजेन्ट चलाउनेछ। हामी `on_message` मेथड प्रयोग गरेर नयाँ सन्देशसँग एजेन्टको अवस्था अपडेट गर्छौं।

यस अवस्थामा, हामी प्रयोगकर्ताबाट आएको नयाँ सन्देश `"Plan me a great sunny vacation"` द्वारा अवस्था अपडेट गर्छौं।

तपाईं सन्देशको सामग्री परिवर्तन गरेर LLM ले कसरी फरक प्रतिक्रिया दिन्छ हेर्न सक्नुहुन्छ।


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छन्। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।  
